In [1]:
import tensorflow as tf
import os
from tensorflow import keras
from tensorflow.python.saved_model import builder as saved_model_builder
from tensorflow.python.saved_model import tag_constants, signature_constants, signature_def_utils_impl
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif,chi2
import numpy as np
import pandas as pd
from keras.callbacks import EarlyStopping
from tensorflow.python.keras import models
from tensorflow.python.keras.layers import Dense
from tensorflow.python.keras.layers import Dropout
import keras.preprocessing.text as kpt
from keras.preprocessing.text import Tokenizer
sess = tf.Session()

/home/exacon02/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


Download Dataset from http://thinknook.com/twitter-sentiment-analysis-training-corpus-dataset-2012-09-22/

In [2]:
training = np.genfromtxt('Sentiment Analysis Dataset.csv', delimiter=',', skip_header=1, usecols=(1, 3), dtype=None,max_rows=100000)

train_x = [x[1] for x in training]
train_y = np.asarray([x[0] for x in training])

/home/exacon02/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: VisibleDeprecationWarning: Reading unicode strings without specifying the encoding argument is deprecated. Set the encoding, use None for the system default.
  """Entry point for launching an IPython kernel.


In [3]:
def mlp_model(layers, units, dropout_rate, input_shape, num_classes):
    """Creates an instance of a multi-layer perceptron model.

    # Arguments
        layers: int, number of `Dense` layers in the model.
        units: int, output dimension of the layers.
        dropout_rate: float, percentage of input to drop at Dropout layers.
        input_shape: tuple, shape of input to the model.
        num_classes: int, number of output classes.

    # Returns
        An MLP model instance.
    """
    model = models.Sequential()
    model.add(Dropout(rate=dropout_rate, input_shape=input_shape))

    for _ in range(layers-1):
        model.add(Dense(units=units, activation='relu'))
        model.add(Dropout(rate=dropout_rate))

    model.add(Dense(units=1, activation='sigmoid'))
    return model

In [4]:
def save_model_for_production(model, version, path='prod_models'):
    tf.keras.backend.set_learning_phase(1)
    if not os.path.exists(path):
        os.mkdir(path)
    export_path = os.path.join(
        tf.compat.as_bytes(path),
        tf.compat.as_bytes(version))
    builder = tf.saved_model.builder.SavedModelBuilder(export_path)

    model_input = tf.saved_model.utils.build_tensor_info(model.input)
    model_output = tf.saved_model.utils.build_tensor_info(model.output)

    prediction_signature = (
        tf.saved_model.signature_def_utils.build_signature_def(
            inputs={'inputs': model_input},
            outputs={'output': model_output},
            method_name=tf.saved_model.signature_constants.PREDICT_METHOD_NAME))

    with tf.keras.backend.get_session() as sess:
        builder.add_meta_graph_and_variables(
            sess=sess, tags=[tf.saved_model.tag_constants.SERVING],
            signature_def_map={
                signature_constants.DEFAULT_SERVING_SIGNATURE_DEF_KEY:prediction_signature,
            })

        builder.save()

In [5]:
# Create keyword arguments to pass to the 'tf-idf' vectorizer.
kwargs = {
            'ngram_range': (1,2),  # Use 1-grams + 2-grams.
            'dtype': 'int32',
            'strip_accents': 'unicode',
            'decode_error': 'replace',
            'analyzer': 'word',  # Split text into word tokens.
            'min_df':  2,
}
vectorizer = TfidfVectorizer(**kwargs)

# Learn vocabulary from training texts and vectorize training texts.
train_x = vectorizer.fit_transform(train_x)

# Select top 'k' of the vectorized features.
selector = SelectKBest(chi2, k=min(20000, train_x.shape[1]))
train_x = selector.fit_transform(train_x, train_y).astype('float32')

In [6]:
# Defining the parameters for the model
layers=2
units=32
dropout_rate=0.3
input_shape=train_x.shape[1:]
num_classes=2


# Create model instance.
model=mlp_model(layers,units,dropout_rate,input_shape,num_classes)

model.compile(loss='binary_crossentropy',
  optimizer=tf.keras.optimizers.Adam(),
  metrics=['accuracy'])

# Train and validate model.
model.fit(train_x, train_y,
  batch_size=512,
  epochs=20,
  verbose=1,
  validation_split=0.2,
  callbacks = [EarlyStopping(monitor='val_loss', patience=1)],     
  shuffle=True)

model.save('model.h5')

Train on 80000 samples, validate on 20000 samples
Epoch 1/20
80000/80000 [==============================] - 14s 181us/step - loss: 0.6461 - acc: 0.6459 - val_loss: 0.5767 - val_acc: 0.7558
Epoch 2/20
80000/80000 [==============================] - 14s 171us/step - loss: 0.5522 - acc: 0.7353 - val_loss: 0.5073 - val_acc: 0.7712
Epoch 3/20
80000/80000 [==============================] - 14s 173us/step - loss: 0.5065 - acc: 0.7587 - val_loss: 0.4790 - val_acc: 0.7821
Epoch 4/20
80000/80000 [==============================] - 14s 172us/step - loss: 0.4801 - acc: 0.7730 - val_loss: 0.4630 - val_acc: 0.7897
Epoch 5/20
80000/80000 [==============================] - 14s 172us/step - loss: 0.4620 - acc: 0.7832 - val_loss: 0.4531 - val_acc: 0.7956
Epoch 6/20
80000/80000 [==============================] - 14s 172us/step - loss: 0.4462 - acc: 0.7913 - val_loss: 0.4434 - val_acc: 0.7993
Epoch 7/20
80000/80000 [==============================] - 14s 172us/step - loss: 0.4351 - acc: 0.7957 - val_loss: 0.

In [10]:
model = keras.models.load_model('model.h5')
while 1:
    evalSentence =input('Input a sentence to be evaluated, or Enter to quit: ')

    if len(evalSentence) == 0:
        break

    # Format your input for the neural net
    evalSentence=[evalSentence]
    tx = vectorizer.transform(evalSentence).astype('float32')
    tx = selector.transform(tx).astype('float32')
    print(tx)
    print(tx.todense())
    pred = model.predict(tx)
    print(pred)
    if pred[0][0]>0.5:
        print("Positive Sentiment.Confidence Level:",(pred[0][0]-0.5)*200,"%")
    else:
        print("Negative Sentiment.Confidence Level:",(0.5-pred[0][0])*200,"%")

Input a sentence to be evaluated, or Enter to quit: i am happy
  (0, 9311)	0.44521123
  (0, 1562)	0.8048806
  (0, 1548)	0.39236996
[[0. 0. 0. ... 0. 0. 0.]]
[[0.9931793]]
Positive Sentiment.Confidence Level: 98.6358642578125 %
Input a sentence to be evaluated, or Enter to quit: i am not happy
  (0, 13315)	0.61494946
  (0, 13261)	0.2730108
  (0, 9311)	0.38012007
  (0, 1573)	0.53905755
  (0, 1548)	0.33500436
[[0. 0. 0. ... 0. 0. 0.]]
[[0.03063744]]
Negative Sentiment.Confidence Level: 93.87251175940037 %
Input a sentence to be evaluated, or Enter to quit: 


In [56]:
#This function saves the model and weights in the .pb format and also freezes and saves the variables
save_model_for_production(model, "1", "/Sentiment")
#Put the Path of the Directory you wish to save your model in.

INFO:tensorflow:No assets to save.
INFO:tensorflow:No assets to write.
INFO:tensorflow:SavedModel written to: /home/exacon02/Models/Sentiment/Test/1/saved_model.pb
